# TGFB1 Part 1 - Importing the data

In this notebook, we take the data from the matrix, gene and cell files and create an AnnData matrix we can use for future analyses.

In [1]:
from collections import defaultdict

import numpy as np
import pandas
import scanpy
import scanpy as sc
import anndata
import matplotlib.pyplot as plt
%matplotlib inline

import numpy as np
import pandas as pd
import scanpy as sc

sc.settings.verbosity = 3             # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.logging.print_versions()
results_file = './write/tgfb1-1.h5ad'  # the file that will store the analysis results

/home/ubuntu/.local/lib/python3.6/site-packages/numba/errors.py:105: UserWarning: Insufficiently recent colorama version found. Numba requires colorama >= 0.3.9
  warnings.warn(msg)


scanpy==1.4 anndata==0.6.19 numpy==1.16.2 scipy==1.2.1 pandas==0.24.2 scikit-learn==0.20.3 statsmodels==0.9.0 python-igraph==0.7.1 louvain==0.6.1 


# Barcode filtering

In order to filter the barcodes, we will open the barcodes done through Cell Ranger and return a set of the barcodes we want to filter the dropest matrix by. It is of the format:

    AAACCTGAGACATAAC-1

where ``AAACCTGAGACATAAC`` is a nucleotide barcode and ``1`` is the batch it corresponds to. We load this into a dictionary mapping barcodes to batches. We will not use the batches in the AnnData matrix, but it is still useful to cross-reference.

In [2]:
def get_barcode_set(path):
    barcodes = {}

    with open("barcodes.tsv", "r") as f:
        for line in f:
            barcode, batch = line.rstrip().split('-')
            batch = int(batch)
            
            barcodes[barcode] = batch
    
    return barcodes

Populate the dictionary from the Cell Ranger set.

In [3]:
pawel_barcodes = get_barcode_set("barcodes.tsv")
len(pawel_barcodes)

12267

# Loading genes

Create a function to load all the gene data from the dimension 1 CSV file, creating a list mapping gene IDs to genes.

The gene file is of the format:

    "1","TGFB1"

where `1` is a strictly increasing gene ID as found in the count matrix and `TGFB1` is the name of the gene. Create a list whose `n`th index is the name of the corresponding gene ID.

In [4]:
def load_genes(batch, variant):
    genes_result = []
    genes_path = "source/{}_{}_dim1.csv".format(batch, variant)

    with open(genes_path, "r") as genes:
        line_no = 0

        for gene in genes:
            if not line_no:
                line_no += 1
                continue

            id, name = gene.rstrip().replace('"', '').split(",")
            id = int(id)
            
            # These should be strictly increasing.
            assert id == line_no

            genes_result.append(name)
            line_no += 1
    
    return genes_result

In [7]:
genes1 = set(load_genes("S1", "exon"))
genes1

{'NARF-AS1',
 'PEX2',
 'KRT33B',
 'RPL5P13',
 'TMEM232',
 'TBR1',
 'PPP1R15B',
 'FAM149B1P1',
 'HNRNPA1P2',
 'AL592494.3',
 'IMPDH1',
 'CACNG8',
 'ZNF726',
 'AC005225.1',
 'SOS1',
 'RNPEP',
 'C4orf17',
 'AL391987.1',
 'PYGB',
 'LINC01635',
 'FBRS',
 'FAM210CP',
 'DNAL4',
 'CTR9',
 'GTSE1',
 'SLC25A6P2',
 'AK4P1',
 'RFX3-AS1',
 'LPIN1',
 'MT-RNR2',
 'DGCR6',
 'AC137932.1',
 'SIGLEC15',
 'AC092418.1',
 'AL078621.2',
 'EFCAB14',
 'MTMR1',
 'RPL23',
 'SOCS6',
 'AC006238.2',
 'EMILIN1',
 'AP000802.1',
 'AL359232.1',
 'AL357078.2',
 'AC087683.1',
 'RNU6-1160P',
 'AC004898.1',
 'AC092953.1',
 'MPZ',
 'CASP16P',
 'RN7SL608P',
 'FIBP',
 'AL137186.2',
 'ZNF14',
 'AC116667.1',
 'NKIRAS2',
 'TTC9C',
 'CYTH2',
 'AC099535.1',
 'PPP1R13B',
 'NAAA',
 'SH2B1',
 'SNX27',
 'MCF2L2',
 'AL356356.1',
 'VAMP8',
 'COG5',
 'AL359091.2',
 'AC024941.1',
 'DHX36',
 'AL022345.4',
 'AC104777.3',
 'LINGO3',
 'SLX1A',
 'PSMF1',
 'CEBPG',
 'TGM4',
 'AL138724.1',
 'AC093525.8',
 'AC007370.1',
 'PACS1',
 'AL445307.1',
 

In [8]:
genes2 = set(load_genes("S2", "exon"))
genes2

{'PEX2',
 'KRT33B',
 'RPL5P13',
 'TMEM232',
 'TBR1',
 'PPP1R15B',
 'AL592494.3',
 'IMPDH1',
 'CACNG8',
 'ZNF726',
 'AC005225.1',
 'SOS1',
 'RNPEP',
 'C4orf17',
 'AL391987.1',
 'PYGB',
 'LINC01635',
 'FBRS',
 'FAM210CP',
 'DNAL4',
 'CTR9',
 'GTSE1',
 'SLC25A6P2',
 'AK4P1',
 'RFX3-AS1',
 'LPIN1',
 'MT-RNR2',
 'DGCR6',
 'AC137932.1',
 'LINC01206',
 'SIGLEC15',
 'AC092418.1',
 'EFCAB14',
 'MTMR1',
 'RPL23',
 'SOCS6',
 'AC006238.2',
 'EMILIN1',
 'AL359232.1',
 'AC087683.1',
 'AC004898.1',
 'CASP16P',
 'AC092953.1',
 'MPZ',
 'RN7SL608P',
 'FIBP',
 'AL137186.2',
 'AC116667.1',
 'ZNF14',
 'NKIRAS2',
 'TTC9C',
 'CYTH2',
 'AC099535.1',
 'PPP1R13B',
 'NAAA',
 'SH2B1',
 'SNX27',
 'MCF2L2',
 'AL356356.1',
 'VAMP8',
 'COG5',
 'AL359091.2',
 'DHX36',
 'AL022345.4',
 'AC104777.3',
 'LINGO3',
 'SLX1A',
 'PSMF1',
 'CEBPG',
 'TGM4',
 'ACOT6',
 'AL138724.1',
 'AC093525.8',
 'AL445307.1',
 'PACS1',
 'MPLKIP',
 'ZNF573',
 'MLXIPL',
 'KLF17',
 'NPIPB3',
 'ERCC6',
 'SLC4A1',
 'SEC61B',
 'MRPS18C',
 'KLHL31',


In [14]:
print(len(genes1), len(genes2), len(genes1) + len(genes2))

28926 28438 57364


In [11]:
symdiff = genes1.symmetric_difference(genes2)
symdiff

{'NARF-AS1',
 'FAM149B1P1',
 'HNRNPA1P2',
 'NFYAP1',
 'WWOX-AS1',
 'LINC01206',
 'AL078621.2',
 'HSPE1P28',
 'AP000802.1',
 'AL357078.2',
 'RNU6-1160P',
 'AC121757.1',
 'AC106779.1',
 'VPS35P1',
 'TPM3P8',
 'KRT8P8',
 'AC026523.2',
 'AC024941.1',
 'TRIM43CP',
 'AC009227.1',
 'AL022324.3',
 'ACOT6',
 'AC007370.1',
 'TBC1D3E',
 'AL590502.1',
 'AC113386.1',
 'AC007954.1',
 'AC069029.1',
 'AL512631.1',
 'DOK2',
 'AL049844.2',
 'CDHR4',
 'AL451142.1',
 'RHCG',
 'LINC01326',
 'AP003110.1',
 'AC073352.1',
 'NMD3P1',
 'CARMIL3',
 'AL080315.1',
 'ALG1L8P',
 'IZUMO2',
 'NDUFB11P1',
 'RNU6ATAC27P',
 'BNIP3P40',
 'RSPO1',
 'VAV1',
 'AL138976.1',
 'TENT5B',
 'SNRPCP9',
 'LINC02108',
 'CSF2',
 'AL353789.1',
 'AL353622.2',
 'PPP1R2P6',
 'SPINT4',
 'AC082651.3',
 'AC005972.2',
 'TNFRSF8',
 'PPIAP90',
 'RF00072',
 'AC084866.2',
 'RNA5SP207',
 'MTRNR2L6',
 'ST6GAL2-IT1',
 'RNU4ATAC18P',
 'AC092839.1',
 'AC110491.3',
 'RN7SL503P',
 'SOX2',
 'FGR',
 'MTTP',
 'TERF1P3',
 'ELFN1',
 'APELA',
 'PPIAP78',
 'AL

In [12]:
len(symdiff)

5418

# Loading barcodes

Create a function to load barcode data from the dimension 2 CSV file. It is of the format:

    "3","AACCGTAATCGGGACCATCATCCC"

where `3` is a strictly increasing cell ID as found in the count matrix, and `AACCGTAATCGGGACCATCATCCC` is the full barcode according to this data set.

Whenever a barcode is encountered, we check if its last 16 nt's have been found in the Cell Ranger barcode set.

This returns a list whose `n`th element corresponds to the barcode of the cell with the corresponding ID, or `None` if the cell ID's barcode is not found in the Cell Ranger barcode set.

In [ ]:
def load_barcodes(batch, variant, good_barcodes):
    result = []
    barcode_path = "source/{}_{}_dim2.csv".format(batch, variant)  
    
    with open(barcode_path, "r") as barcodes:
        # Skip R crap in first line.
        next(barcodes)
        
        for i, line in enumerate(barcodes):
            cell_id, full_barcode = line.rstrip().replace('"', '').split(",")
            barcode = full_barcode[-16:]
            cell_id = int(cell_id)
            
            # The cell IDs should be continuous.
            assert cell_id == (i + 1)
            
            if barcode not in good_barcodes:
                result.append(None)
            elif good_barcodes[barcode] != int(batch[-1]):
                result.append(None)
                #print("ERROR:", barcode, "is in good set but corresponds to batch", good_barcodes[barcode])
            else:
                result.append(barcode)
    
    return result

As an experiment, create a histogram to see how many cell IDs appear per cellular barcode. We can see that all of them except one have 4 cell IDs corresponding to a barcode, the rest of the barcode is the sample barcode.

In [ ]:
# Uncomment to run experiment.

"""
barcodes = load_barcodes("S1", "exon", pawel_barcodes)
histogram = defaultdict(int)

for barcode in barcodes:
    if barcode:
        histogram[barcode] += 1

histogram
"""

# Loading transcript count matrix

The count matrix is of the format:

    %
    no_genes no_cells no_counts
    19 1 1
    34 1 1
    [...]
    33663 12318 1
    33665 12318 35
    33678 12318 1

where the first number represents the gene, the second number represents the cell, and the third number represents the count.

We then get a `cells` by `genes` count matrix indexed by the cell IDs and gene IDs as described above.

In [ ]:
def load_transcript_matrix(batch, variant, genes, barcodes):
    matrix_path = "source/{}_{}.mmf".format(batch, variant)
    
    matrix = None
    
    with open(matrix_path, "r") as f:
        first_line = True
        
        for line in f:
            if "%" in line:
                continue
            
            gene, cell, count = [int(i) for i in line.rstrip().split()]
            
            if first_line:
                assert gene == len(genes)
                assert cell == len(barcodes)
                
                first_line = False
                matrix = np.ndarray(shape=(cell, gene), dtype=np.dtype(np.int16))
                continue
            
            
            gene_pos = gene - 1
            cell_pos = cell - 1
            
            matrix[cell_pos, gene_pos] = count
    
    return matrix

# Create Pandas DataFrame from count matrix

Given a count matrix of size `genes x cells`, a list of length `genes` and a list of length `cells`, create a Pandas DataFrame with index `cells` and column labels from `genes`.

As we have seen from the barcode loading function, we will have four rows in our table corresponding to the same barcode, and some of the data we get from the matrix does not correspond to a barcode in the Cell Ranger set. During this process, we will filter out all rows that do not have a valid Pawel barcode (i.e. index is None), and group and sum all (4) rows that have the same barcode.

In [ ]:
def dataframe_from_matrix(m, genes, barcodes):
    assert m.shape == (len(barcodes), len(genes))
    
    # Create a data frame from the count matrix, keyed by the barcode
    # and gene annotations we created.
    result = pandas.DataFrame(m)
    result.index = barcodes
    result.columns = genes
    
    # First, get rid of all the rows whose barcode is None. This means those
    # cells were not found in the reference barcode set.
    result = result[result.index.notnull()]
    
    # Then, since each barcode appears to correspond to four cell IDs,
    # we get counts spread out across these four IDs. Let's collapse the duplicates.
    result = result.groupby(by=lambda x: x).sum()
    
    return result

# Construct DataFrame

Given a batch identifier list (i.e. `["S1", "S2"]`) and matrix name list (i.e. `["exon", "intron"]`), create a Pandas DataFrame from the corresponding count matrices and gene/cell dimension file for each. In addition, update a set `genes_set` with all genes that have been encountered in this count matrix, as these are not normalized by default across matrices in a batch (unlike cell IDs).

In [ ]:
def construct_dataframes(batches, matrices, genes_set):
    for batch_name in batches:    
        prev_barcodes = None

        for matrix_name in matrices:
            print("Now processing", batch_name, matrix_name)

            genes = load_genes(batch_name, matrix_name)        
            print("Genes list loaded at", len(genes), "items.")

            barcodes = load_barcodes(batch_name, matrix_name, pawel_barcodes)
            print("Barcodes list loaded at", len(barcodes), "items.")

            print("Loading transcription matrix and converting to dataframe...")
            m = load_transcript_matrix(batch_name, matrix_name, genes, barcodes)
            m = dataframe_from_matrix(m, genes, barcodes)

            genes_set.update(m.columns)
            print("Total gene set is now", len(genes_set), "long.")

            if prev_barcodes is not None:
                assert m.index.equals(prev_barcodes)
            else:
                prev_barcodes = m.index

            m.to_pickle("/tmp/{}_{}.pickle".format(batch_name, matrix_name))
            print("Count matrix with dimensions", m.shape, "created and pickled for", batch_name, matrix_name)
            print("")

        print("")

# Normalize genes in DataFrames

Given a batch identifier list (i.e. `["S1", "S2"]`) and matrix name list (i.e. `["exon", "intron"]`), load the corresponding DataFrames. Take the `genes_set` generated from the previous process and add to each dataframe the genes that are not encountered in that count matrix, and the counts to 0 for all cells. This is needed as all count matrices need to be the same dimension when the AnnData layers are generated. To normalize the list of genes, we will sort the columns alphabetically as well.

In [ ]:
def normalize_dataframe_genes(batches, matrices, genes_set):
    print("Normalizing dimensions of count matrices along genes...")

    for batch_name in batches:    
        for matrix_name in matrices:
            m = pandas.read_pickle("/tmp/{}_{}.pickle".format(batch_name, matrix_name))

            print("Old shape of", batch_name, matrix_name, "was", m.shape)
            missing_genes = genes_set - set(m.columns)
            for missing_gene in missing_genes:
                m[missing_gene] = 0
            print("New shape is", m.shape)

            print("Reordering genes in alphabetical order...")
            print("Old column order was", m.columns)
            m = m.reindex(sorted(m.columns), axis=1)
            print("New column order is", m.columns)

            m.to_pickle("/tmp/{}_{}.pickle".format(batch_name, matrix_name))
            print("Normalized count matrix for", batch_name, matrix_name, "pickled.")
            print("")

        print("")

# Create layered AnnData object from batch matrices

Create a layered AnnData object from frames for each individual batch. In practice, this takes all frames with names in a matrix list (i.e. `["intron", "exon", "spanning"]`) for each batch (i.e. `["S1", "S2"]`), and creates a layered AnnData object with layer names corresponding to `layer_names` (i.e. `["unspliced", "spliced", "ambiguous"]`) from these frames.

Frames should have identical dimensions, indices and column names for this to work.

The layer with name `spliced` is implicitly used as the main count matrix of the AnnData object.

In [ ]:
def create_anndata_from_frames(batches, matrices, layer_names):
    results = []
    
    for batch in batches:
        result = None
        print("Now processing batch", batch, "...")
        
        for matrix, layer in zip(matrices, layer_names):
            print("Loading matrix", matrix, "...")
            m = pandas.read_pickle("/tmp/{}_{}.pickle".format(batch, matrix))
            
            if not result:
                print("Creating AnnData object...")
                
                result = anndata.AnnData(shape=m.shape)
                result.obs = pandas.DataFrame({"cellular_barcode": list(m.index)})
                result.var = pandas.DataFrame({"gene_names": list(m.columns)})
                
            if layer == "spliced":
                print("Layer is", layer, ", adding it as main count matrix to object...")
                result.X = m
            
            print("Adding count matrix as layer", layer, "to AnnData object...")
            result.layers[layer] = m.values
            print("")

        result.var_names = [gene for gene in result.var["gene_names"]]
        result.obs_names = [barcode for barcode in result.obs["cellular_barcode"]]
        result.obs_names_make_unique()
        results.append(result)
                
        print("Final AnnData object is", result)
        
        print("Saving layered AnnData object for batch", batch, "...")
        result.write("./write/{}-final.h5ad".format(batch))
        print("")

# Merge AnnData objects along batches

Given a list of AnnData objects corresponding to each batch, create one large AnnData object comprising all batches. Save merged AnnData object for future processing.

In [ ]:
def create_merged_anndata(batches):
    result = None
    to_merge = []
    
    for batch in batches:
        if not result:
            print("Merged AnnData file not yet created, making it from batch", batch)
            result = anndata.read_h5ad("./write/{}-final.h5ad".format(batch))
            print("Merged AnnData file is now", result)
        else:
            print("Loading batch", batch)
            curr = anndata.read_h5ad("./write/{}-final.h5ad".format(batch), backed="r+")
            print("Loaded AnnData file is", curr, ", appending to list of objects to concatenate...")
            to_merge.append(curr)
        print("")
    
    print("")        
    print("Concatenating list", to_merge, "to original file")
    result = result.concatenate(*to_merge, batch_categories=batches, index_unique=None)
    print("Concatenated AnnData object is", result, ", saving it to disk...")
    result.write("./write/merged-final.h5ad")

# Run the procedures above
Create the variables for this experiment and run the procedures above, creating an AnnData matrix used in future notebooks.

In [ ]:
batches = ('S1', 'S2')
matrices = ('exon', 'intron', 'spanning')
layer_names = ('spliced', 'unspliced', 'ambiguous')
all_genes = set()

In [ ]:
construct_dataframes(batches, matrices, all_genes)

In [ ]:
normalize_dataframe_genes(batches, matrices, all_genes)

In [ ]:
create_anndata_from_frames(batches, matrices, layer_names)

In [ ]:
create_merged_anndata(batches)